**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_ENCODE_FCC/references 



**Set global variables**

In [2]:
TXT_FOLDER_REGION = "hic_intact_K562_ENCSR479XDG"
TXT_ASSAY = "hic_intact"
TXT_INDEX = "ENCSR479XDG"

## Import data

In [3]:
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_DAT, "external", txt_folder)

vec = dir(txt_fdiry)
for (txt in vec){cat(txt, "\n")}

K562.hg38.ENCSR479XDG.ENCFF126GED.hic_intact.contact_domain.bedpe.gz 
K562.hg38.ENCSR479XDG.ENCFF256ZMD.hic_intact.loop.bedpe.gz 
K562.hg38.ENCSR479XDG.ENCFF621AIY.hic_intact.matrix.hic 
run_download.log.txt 
run_download.sh 


In [4]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_DAT, "external", txt_folder)
txt_fname = "K562.hg38.ENCSR479XDG.ENCFF126GED.hic_intact.contact_domain.bedpe.gz"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read table
dat = read_tsv(txt_fpath, show_col_types = FALSE)

### arrange columns
dat = dat %>% dplyr::filter(!row_number() == 1)
colnames(dat)[1] = "chr"

### assign and show
dat_region_import = dat
print(dim(dat))
fun_display_table(head(dat))

New names:
• `score` -> `score...8`
• `score` -> `score...12`
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details, e.g.:
  dat <- vroom(...)
  problems(dat)”


[1] 4348   16


chr,x1,x2,chr2,y1,y2,name,score...8,strand1,strand2,color,score...12,uVarScore,lVarScore,upSign,loSign
chr10,89790000,90730000,chr10,89790000,90730000,.,.,.,.,2552550,0.4873343,0.4321875,0.3910142,0.4366181,0.5132139
chr10,117485000,118260000,chr10,117485000,118260000,.,.,.,.,2552550,0.4714600,0.4038806,0.4028087,0.4469099,0.4451019
chr10,51125000,52215000,chr10,51125000,52215000,.,.,.,.,2552550,0.5757076,0.3855988,0.3954850,0.4712260,0.4000834
chr10,112495000,113565000,chr10,112495000,113565000,.,.,.,.,2552550,0.6096007,0.3796908,0.3952655,0.5317584,0.4613188
chr10,84540000,86085000,chr10,84540000,86085000,.,.,.,.,2552550,0.5786491,0.4119491,0.3597105,0.4424142,0.4804162
chr10,94665000,95075000,chr10,94665000,95075000,.,.,.,.,2552550,0.6267519,0.3811024,0.3788707,0.5000000,0.4018583


In [5]:
colnames(dat)

[1] "chr"        "x1"         "x2"         "chr2"       "y1"        
 [6] "y2"         "name"       "score...8"  "strand1"    "strand2"   
[11] "color"      "score...12" "uVarScore"  "lVarScore"  "upSign"    
[16] "loSign"

## Arrange loops

In [6]:
### init
dat = dat_region_import
vec_txt_cname1 = c(
    "chr",  "x1", "x2", 
    "chr2", "y1", "y2")
vec_txt_cname2 = c(
    "Chrom_A", "ChromStart_A", "ChromEnd_A",
    "Chrom_B", "ChromStart_B", "ChromEnd_B"
)
vec_txt_cname3 = c(
    "Chrom_A", "ChromStart_A", "ChromEnd_A", "Region_A", "Length_A",
    "Chrom_B", "ChromStart_B", "ChromEnd_B", "Region_B", "Length_B",
    "Name", "Size"
)

### get columns
dat = dat %>% dplyr::select(!!!vec_txt_cname1)
colnames(dat) = vec_txt_cname2

### arrange columns
dat = dat %>% 
    dplyr::mutate(
        Length_A = ChromEnd_A - ChromStart_A,
        Length_B = ChromEnd_B - ChromStart_B,
        Size     = (Length_A + Length_B) / 2,
        Region_A   = paste0(Chrom_A, ":", ChromStart_A, "-", ChromEnd_A),
        Region_B   = paste0(Chrom_B, ":", ChromStart_B, "-", ChromEnd_B),
        Name       = paste0(Region_A, "|", Region_B)
    ) %>%
    dplyr::select(!!!vec_txt_cname3)

### assign and show
dat_region_arrange = dat
print(dim(dat))
fun_display_table(head(dat))

[1] 4348   12


Chrom_A,ChromStart_A,ChromEnd_A,Region_A,Length_A,Chrom_B,ChromStart_B,ChromEnd_B,Region_B,Length_B,Name,Size
chr10,89790000,90730000,chr10:89790000-90730000,940000,chr10,89790000,90730000,chr10:89790000-90730000,940000,chr10:89790000-90730000|chr10:89790000-90730000,940000
chr10,117485000,118260000,chr10:117485000-118260000,775000,chr10,117485000,118260000,chr10:117485000-118260000,775000,chr10:117485000-118260000|chr10:117485000-118260000,775000
chr10,51125000,52215000,chr10:51125000-52215000,1090000,chr10,51125000,52215000,chr10:51125000-52215000,1090000,chr10:51125000-52215000|chr10:51125000-52215000,1090000
chr10,112495000,113565000,chr10:112495000-113565000,1070000,chr10,112495000,113565000,chr10:112495000-113565000,1070000,chr10:112495000-113565000|chr10:112495000-113565000,1070000
chr10,84540000,86085000,chr10:84540000-86085000,1545000,chr10,84540000,86085000,chr10:84540000-86085000,1545000,chr10:84540000-86085000|chr10:84540000-86085000,1545000
chr10,94665000,95075000,chr10:94665000-95075000,410000,chr10,94665000,95075000,chr10:94665000-95075000,410000,chr10:94665000-95075000|chr10:94665000-95075000,410000


**Check**

In [7]:
dat  = dat_region_arrange

vec1 = dat$Region_A
vec2 = dat$Region_B
print(all(vec1 == vec2))

vec1 = dat$Length_A
vec2 = dat$Length_B
print(all(vec1 == vec2))

[1] TRUE
[1] TRUE


## Prepare region

In [9]:
### init
vec_txt_cname_A = c("Chrom_A", "ChromStart_A", "ChromEnd_A", "Region_A")
vec_txt_cname   = c("Chrom",   "ChromStart",   "ChromEnd",   "Region", "Group", "Label")
txt_assay = TXT_ASSAY

### prepare region
dat = dat_region_arrange
dat = dat %>% 
    dplyr::select(!!!vec_txt_cname_A) %>% 
    dplyr::mutate(
        Group = txt_assay,
        Label = "TAD"
    ) %>%
    dplyr::distinct()
colnames(dat) = vec_txt_cname

### assign and show
dat_region_simplify = dat
print(dim(dat))
fun_display_table(head(dat))

[1] 4348    6


Chrom,ChromStart,ChromEnd,Region,Group,Label
chr10,89790000,90730000,chr10:89790000-90730000,hic_intact,TAD
chr10,117485000,118260000,chr10:117485000-118260000,hic_intact,TAD
chr10,51125000,52215000,chr10:51125000-52215000,hic_intact,TAD
chr10,112495000,113565000,chr10:112495000-113565000,hic_intact,TAD
chr10,84540000,86085000,chr10:84540000-86085000,hic_intact,TAD
chr10,94665000,95075000,chr10:94665000-95075000,hic_intact,TAD


## Save results

In [10]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder)
txt_fname = "K562.hg38.hic_intact.TAD.bedpe.gz"
txt_fpath = file.path(txt_fdiry, txt_fname)

### get table
dat = dat_region_simplify
dat = dat %>% dplyr::arrange(Chrom, ChromStart, ChromEnd)

### write tabel
dir.create(txt_fdiry, showWarnings = FALSE)
write_tsv(dat, txt_fpath, col_names = FALSE)

In [11]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder, "summary")
txt_fname = "K562.hg38.hic_intact.TAD.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

### get table
dat = dat_region_arrange
dat = dat %>% dplyr::arrange(
    Chrom_A, ChromStart_A, ChromEnd_A,
    Chrom_B, ChromStart_B, ChromEnd_B
)

### write tabel
dir.create(txt_fdiry, showWarnings = FALSE)
write_tsv(dat, txt_fpath)